## Importação de dados e biblioteca

In [1]:
import pandas as pd

df = pd.read_csv('ponderada4/backend/BrazilTotalHighwayCrashes.csv')

<ipython-input-1-521e2489a20a>:3: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/BrazilTotalHighwayCrashes.csv')


In [2]:
df

,data,horario,n_da_ocorrencia,tipo_de_ocorrencia,km,trecho,sentido,lugar_acidente,tipo_de_acidente,automovel,...,outros,tracao_animal,transporte_de_cargas_especiais,trator_maquinas,utilitarios,ilesos,levemente_feridos,moderadamente_feridos,gravemente_feridos,mortos
0,01/01/2010,04:21:00,18,sem vítima,167,BR-393/RJ,Norte,Rodovia do Aço,Derrapagem,1,...,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0
1,01/01/2010,02:13:00,20,sem vítima,"269,5",BR-116/PR,Sul,Autopista Regis Bittencourt,Colisão Traseira,2.0,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
2,01/01/2010,03:35:00,000024/2010,sem vítima,77,BR-290/RS,Norte,Concepa,COLISÃO LATERAL,2.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
3,01/01/2010,07:31:00,000038/2010,sem vítima,52,BR-116/RS,Norte,Concepa,QUEDA DE MOTO,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,01/01/2010,04:57:00,000027/2010,sem vítima,33,BR-290/RS,Norte,Concepa,QUEDA DE MOTO,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81465,03/11/2018,00:07:00,1,ac02 - Acidente com Vítima,396,BR-262/MG,Leste,Concebra,Capotamento,1.0,...,NaN,NaN,NaN,NaN,NaN,0.0,3.0,NaN,NaN,NaN
81466,03/11/2018,22:18:00,179,sem vítima,"200,7",BR-101/RJ,Sul,Autopista Fluminense,Atropelamento de Animal,1.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
81467,03/11/2018,20:11:00,166,sem vítima,"144,4",BR-101/RJ,Sul,Autopista Fluminense,Choque - Elemento de Drenagem,1.0,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
81468,03/11/2018,18:41:00,394,com vítima,"613,536",BR-163/MS,Norte,MSVIA,Colisão com veículos (frontal),1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0


## Cria nova coluna gravidade de acidentes

In [3]:
# A gravidade do acidente aumenta de acordo com os feridos do acidente

df["gravidade"] = (
    df["levemente_feridos"] +
    df["moderadamente_feridos"] * 2 +
    df["gravemente_feridos"] * 3 +
    df["mortos"] * 4
)

In [4]:
# Elimina linha que possuem valores nulos

df = df.dropna(subset=["gravidade"])

In [5]:
# Verifica valores existente de gravidade

df['gravidade'].value_counts()

0.0     17650
1.0      3985
2.0      2534
3.0       775
4.0       672
5.0       172
6.0       143
7.0        64
8.0        53
10.0       31
9.0        24
11.0       15
14.0       15
12.0       12
13.0       10
17.0        7
15.0        7
18.0        7
16.0        6
20.0        3
21.0        3
22.0        3
23.0        2
40.0        2
52.0        1
26.0        1
19.0        1
25.0        1
28.0        1
29.0        1
Name: gravidade, dtype: int64

In [6]:
# Classifica em Leve, Moderado, Significativo e Grave os acidentes (Esse será o valor que o modelo tentará classificar)

def aplicar_metrica(valor):
    if valor == 0.0:
        return 'Leve'
    elif valor <= 10.0 and valor > 0.0:
        return 'Moderado'
    elif valor > 10.0 and valor <= 20.0 :
        return 'Significativo'
    elif valor > 20.0:
        return 'Grave'

df['gravidade_label'] = df['gravidade'].apply(aplicar_metrica)

<ipython-input-6-99201aba53ae>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gravidade_label'] = df['gravidade'].apply(aplicar_metrica)


## Remove colunas não utilizadas

In [7]:
df.drop(['automovel',	'bicicleta',	'caminhao',	'moto',	'onibus',	'outros',	'tracao_animal',	'transporte_de_cargas_especiais',	'trator_maquinas', 'lugar_acidente','gravidade', 'ilesos', 'levemente_feridos', 'moderadamente_feridos', 'gravemente_feridos', 'mortos', 'km', 'data', 'horario', 'n_da_ocorrencia', 'utilitarios'], axis=1, inplace=True)

<ipython-input-7-a0425a5973c1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['automovel',	'bicicleta',	'caminhao',	'moto',	'onibus',	'outros',	'tracao_animal',	'transporte_de_cargas_especiais',	'trator_maquinas', 'lugar_acidente','gravidade', 'ilesos', 'levemente_feridos', 'moderadamente_feridos', 'gravemente_feridos', 'mortos', 'km', 'data', 'horario', 'n_da_ocorrencia', 'utilitarios'], axis=1, inplace=True)


In [8]:
df

,tipo_de_ocorrencia,trecho,sentido,tipo_de_acidente,gravidade_label
0,sem vítima,BR-393/RJ,Norte,Derrapagem,Leve
2,sem vítima,BR-290/RS,Norte,COLISÃO LATERAL,Leve
3,sem vítima,BR-116/RS,Norte,QUEDA DE MOTO,Leve
4,sem vítima,BR-290/RS,Norte,QUEDA DE MOTO,Leve
5,sem vítima,BR-290/RS,Sul,SAÍDA DE PISTA,Leve
...,...,...,...,...,...
81421,Acidente sem vítima,BR-101/ES,Norte,Colisão Lateral,Leve
81422,Acidente sem vítima,BR-101/ES,Sul,Colisão Lateral,Leve
81425,sem vítima,BR-116/PR,Sul,"Choque - Defensa, barreira ou submarino",Leve
81439,Acidente sem vítima,BR-101/ES,Sul,Choque com Objeto Fixo,Leve


## Arruma colunas que possuem valores igual com tipografia diferente

#### Coluna 'Tipo de concorrencia"

In [9]:
# Possiveis valores registrados

df['tipo_de_ocorrencia'].value_counts()

sem vítima                                   6774
Sem vítima                                   5263
com vítima                                   3689
Acidente sem Vítima                          2178
Acidente sem vítima                          1803
Com vítima                                   1765
Acidente com vítima                          1588
Acidente com Vítima                           886
Acidente sem Vítimas                          524
Acidente com Vítimas                          323
Acidente sem VITIMA                           312
Acidente com Danos Materiais                  273
Acidente com VITIMA                           168
ac02 - Acidente com Vítima                    119
Atropelamento sem Morte                       118
Atropelamento sem morte                        77
Acidente com Morte                             74
Acidente com morte                             58
ac03 - Acidente com Danos Materiais            32
Acidente Fatal                                 28


In [10]:
# Mudar tipografia de valores existentes para um padrão único

sem_vitima = [ 'Sem vítima','AC03 - Acidente sem VITIMA','sem vítima', 'Acidente sem Vítima', 'Acidente sem vítima', 'Acidente sem Vítimas', 'Acidente sem VITIMA', 'Atropelamento sem Morte', 'Atropelamento sem morte', 'Acidente sem Vítima - TRECHO CONTORNO', 'Acidente sem vítima - TRECHO CONTORNO', 'Atropelamento sem Morte - TRECHO CONTORNO', 'Acidente sem vítima - TRECHO CONTORNO']
com_vitima = [ 'Acidente com vítima','Atropelamento com morte','Atropelamento com morte ','AC02 - Acidente com VITIMA','Acidente com VITIMA FATAL', 'Atropelamento','ac03 - Acidente com vítima ilesa','Atropelamento Fatal','Acidente com Vítima - TRECHO CONTORNO' ,'Acidente com vítima - TRECHO CONTORNO', 'com vítima', 'Acidente com Vítima', 'Acidente com Vítima', 'Acidente com Vítima', 'Acidente com Vítimas', 'Acidente com VITIMA', 'Acidente COM VITIMA', 'Com vítima', 'ac02 - Acidente com Vítima', 'ac02 - Acidente com vitima', 'ac02 - Acidente com vítima', 'ac02 - Acidente com vítima',  'Atropelamento sem Morte', 'Atropelamento sem morte', 'Atropelamento sem morte - TRECHO CONTORNO','Acidente com Morte', 'Acidente com morte', 'Acidente com Morte - TRECHO CONTORNO', 'Acidente com morte - TRECHO CONTORNO', 'Acidente fatal', 'Atropelamento com Morte', 'Acidente Fatal', 'AF - Acidente com Vitima Fatal', 'ac01 - Acidente com Vítima Fatal', 'ac01 - Acidente com vitima fatal', 'Acidente com Vítima Fatal', 'AC01 - ACIDENTE COM VÍTIMA FATAL', '1 - Acidente com Vitima Fatal']
danos_materias = [ 'ac03 - Acidente com Danos Materiais', 'Acidente com Danos Materiais']

def categorizar_acidente(valor):
    if valor in sem_vitima:
        return "Sem vítimas"
    elif valor in com_vitima:
        return "Com vítimas"
    elif valor in danos_materias:
        return "Danos materiais"
    else:
        return valor

df['tipo_de_ocorrencia'] = df['tipo_de_ocorrencia'].apply(categorizar_acidente)


<ipython-input-10-70b85cd62717>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tipo_de_ocorrencia'] = df['tipo_de_ocorrencia'].apply(categorizar_acidente)


In [11]:
# Valores registrados depois do tratamento

df['tipo_de_ocorrencia'].value_counts()

Sem vítimas        17075
Com vítimas         8821
Danos materiais      305
Name: tipo_de_ocorrencia, dtype: int64

#### Coluna 'tipo_de_acidente'

In [12]:
# Possiveis valores registrados

df['tipo_de_acidente'].value_counts()

Colisão traseira                     4388
Colisão Traseira                     1407
Capotamento                          1375
Engavetamento                        1342
Abalroamento longitudinal            1232
                                     ... 
Choque- Placa/Suporte Sinalização       1
COLISÃO FRONTAL                         1
Ablr.Sentido Oposto                     1
Atropelamento com morte                 1
Choque - New jersey                     1
Name: tipo_de_acidente, Length: 183, dtype: int64

In [13]:
# Mudar tipografia de valores existentes para um padrão único

# Colisões
colisoes = [
    'Derrapagem', 'COLISÃO LATERAL', 'COLISÃO TRASEIRA', 'Colisão traseira',
    'Colisão Frontal', 'Abalroamento longitudinal', 'Colisão Transversal',
    'Abalroamento - Longitudinal', 'Abalroamento - Transversal',
    'Colisão - Traseira', 'Colisão - Lateral', 'Colisão - Frontal',
    'Colisão - Transversal', 'Colisão', 'Colisão Lateral Mesmo Sentido',
    'Colisão Ojeto Fixo', 'Colisão Objeto Fixo', 'Colisão com Veículo Estac.', 'Colisão Traseira', 'Colisão Frontal', 'Colisão Frontal ', 'Colisão com veículo estacionado',
    'Choque na Praça - Cabine', 'Colisão lateral no mesmo sentido', 'Colisão Lateral', 'Colisão transversal', 'Colisão frontal', 'Colisão com Objeto Fixo',
    'Colisão c/ obstáculos', 'Colisão com veículos (traseira)', 'Colisão lateral no sentido contrário',  'Colisão com veículos (frontal)', 'Choque Barreira', 'Colisão lateral', 'Colisão lateral em sentido contrário', 'COLISÃO COM OBSTÁCULO',
    'Colisão com ciclista', 'Colisão com objeto fixo', 'colisão traseira',  'COLISÃO FRONTAL', 'Colisão com Objeto fixo', 'Colisão objeto fixo', 'Colisão com objeto Fixo', 'COLISÃO TRANSVERSAL',
    'colisão lateral', 'colisão com obstáculo', 'colisão Frontal', 'Colisão com veículo da concessionária' ,  'colisão frontal',
    'Colisão com Obstáculos', 'Colisão Lateral - sentido contrário', 'Colisão Lateral - mesmo sentido', 'Colisão Traseira - veículos', 'Colisão na Praça - Submarino', 'Colisão Frontal - veículos', 'Colisão Lateral Sentido Oposto', 'Colisão na Praça - Cancela', ' INCIDENTE - Colisão na Praça - Cancela','Colisão veiculo parado acostamento'


]

# Quedas
quedas = [
    'QUEDA DE MOTO', 'Queda de ribanceira', 'Queda de Moto', 'Queda de ciclista',
    'Queda de Carga', 'Queda de bicicleta', 'Queda Veículo', 'QUEDA DE CARGA',
    'Queda em Ribanceira', 'Queda de moto',  'Queda de Bicicleta', 'Queda de Ciclista','Queda de Ponte/Viaduto',  'queda de moto',  'Queda de Viaduto', 'Queda de Ribanceira',  'Queda de carga', 'queda de carga',
    'Queda de ponte/viaduto', 'Queda de veículo em ribanceira', 'Queda de Ponte / Viaduto', 'Queda de Carro', 'Queda de vegetação c/ veículo', 'Queda de barreira, ribanceira, ponte ou viaduto', 'Queda de veículo em ribanceira, ponte ou viaduto'
]

# Saídas de Pista
saidas_de_pista = [
    'SAÍDA DE PISTA', 'Saída de pista', 'Saida de Pista', 'Saída de Pista ',
    'Saídas de pista', 'Sem vítimas', 'saída de Pista', 'Saída de Pista',  'Saida de pista',  'Saída de  Pista', 'saída de pista'
]

# Atropelamentos
atropelamentos = [
    'Atropelamento de pedestre atravessando', 'Atropelamento de animal',
    'Atropelamento de pedestre caminhando', 'Atropelamento sem morte',
    'Atropelamento - Pedestre atravessando', 'Atropelamento de animais',
    'Atropelamento - Pedestre caminhando', 'Atropelamento - Equino',
    'Atropelamento de Ciclista', 'Atropelamento de Pedestre',
    'Atropelamento - Ciclista', 'Atropelamento - Morador',
    'Atropelamento - Bovino', 'Atropelamento de Bovino', 'Atropelamento Animal',
    'Atropelamento - Abrigo de ônibus', 'Atropelamento em abrigo de ônibus',
    'Atropelamento - Morador/Trabalhador/Estudante', 'Atropelamento - Esportista',
    'Atropelamento - Pedestre caminhando', 'Atropelamento - Pedestre atravessando',
    'Atropelamento - Suicida', 'Atropelamento - Morador/Trabalhador',
    'Atropelamento de Pedestre', 'Atropelamento - Andarilho', 'Atropelamento - Sem Informação',
    'Atropelamento - Bubalino', 'Atropelamento de animais', 'Atropelamento de Andarilho',
    'Atropelamento  de Pedestre', 'atropelamento de animais', 'Sem vítimas', 'Atropelamento - Outros', 'atropelamento', 'Atropelamento com morte', 'Atropelamento de Animal','Atropelamento', 'Atropelamento - Animal', 'Atropelamento e Fuga',
    'Atropelamento de pedestre', 'Atropelamento de Equino',
     'Atropelamento - Funcionário', 'Atropelamento de ciclista',  'Atropelamento - Usuário', 'Atropelamento - Pedestre', 'ATROPELAMENTO DE ANIMAIS', 'ATROPELAMENTO DE PEDESTRE',  'ATROPELAMENTO ANIMAIS', 'atropelamento de pedestre',
     'Atropelamento de Animal ', 'Atropelamento - Ambulante',  'Atropelamento Ciclista',   'ATROPELAMENTO PEDESTRE', 'Atropelamento ',
     'Atropelamento de Cachorro', 'Atropelamento de Pedestre Atravessando',  'Atropel. de pedestre atravessando',  'Atropelamento de Pedestre Caminhando', 'Atropelamento - Pessoa', 'Atropelamento - Cachorro', 'Atropel. de pedestre caminhando', 'Atropelamento - Esportista/Romeiro','Atropelamento - Pedestre Andarilho',  'Atropelamento - Pedestre Usuário', 'Atropelamento - PedestreFuncionário'


]

# Choques e Colisões com Objetos
choques_e_colisoes_com_objetos = [
    'Choque em barreira New Jersey', 'Choque em objeto fixo', 'Choque em objeto na pista',
    'Choque em defensa', 'Choque - Poste', 'Choque - Defensa', 'barreira ou submarino',
    'Choque - Cancela de Pedagio', 'Choque - Objeto sobre a pista', 'Choque com objeto',
    'Choque c/ Objeto Fixo', 'Choque - Defensa', 'Choque - Talude', 'Choque - Arvore',
    'Choque - Defensa metálica', 'Choque - Painel Propaganda', 'Choque- Placa/Suporte Sinalização',
    'Choque / objeto não identificado', 'Choque - Objeto Fixo', 'Choque com objeto sobre a pist',
    'Choque contra Objeto Fixo', 'Choque com objeto na pista', 'Choque - Cabine de Pedágio',
    'Choque na cancela', 'Choque - Submarino', 'Choque / defensa, barreira ou "submarino"',
    'Choque / árvore', 'Choque - Canaleta', 'Choque - Veiculo parado na pista',
    'CHOQUE C/ OBJETO NA PISTA', 'Choque veículo parado na pista', 'Choque com veículo na faixa de rolamento',
    'Choque - Veiculo parado no acostamento', 'Choque- Poste/Equipamento', 'Choque -', 'Choque na Praça - Cabine ',
    'Com vítimas', 'Danos materiais', 'Choque Talude', 'Não Def', 'Choque', 'Choque - Defensa, barreira ou  submarino', 'Choque com objeto fixo', 'Choque com Objeto Fixo',
    'Choque em veículo parado na pista',  'Choque - Defensa, barreira ou  submarino ', 'Choque - Elemento de Drenagem', 'Choque - Suporte de Sinalização',
     'Choque - Outros', 'Choque - Defensa, barreira ou "submarino"', 'Choque - Pilar',
     'Choque Defensa', 'Choque - Meio Fio','Choque com objeto sobre a pista',  'Choque com Veículo Estacionado', 'Choque Praça - Cabine',
     'Choque - Sinalização', 'Choque - Barreira',  'Choque com veículo no acostamento', 'Choque - New jersey', 'Choque com veículo na faixa de', 'Choque Praça - Submarino',  'Choque c/ Veículo Parado', 'Choque Praça',
      'Choque contra objeto na faixa de rolamento', 'Choque - Objeto não identificado', 'Choque - Caixa de Captação/Fibra', 'Choque com veículo da concessionária estacionado', 'Choque - Talude ou Barranco',
    'Choque em Barreira New Jersey', 'Choque em Defensa', 'Choque - Defensa, barreira ', 'Choque - Barranco',
    'Choque - Defensa, barreira ou meio fio', 'Choque - Meio fio', 'choque em objeto fixo', 'Choque em barreira New Jersey ', 'Choque Contra Objeto Fixo', 'Choque em Objeto Fixo', 'Choque contra veículo no acostamento', 'Choque em Objeto fixo', 'Choque - Obstáculo Fixo', 'Choque na praça ? submarino', 'Choque - Poste/', 'Choque - Praça', 'Choque na Praça - Submarino', 'choque - Objeto Fixo', 'Choque - Veículo parado na pista', 'Choque - Objeto na Pista', 'Choque-outros', 'choque c/ objeto na pista',



]

# Lista com o nome "abalroamento" ou "abalr."
lista_abalroamento = [
    'Abalroamento - Mesmo Sentido', 'Abalr.Mesmo Sentido', 'Abal.Transversal', 'Abal.sent. Oposto',
    'Abal.Transversal', 'Abal.mesmo.sentido', 'Abalr. Sentido Oposto', 'Abalroamento - Transversal',
    'Abal.mesmo Sentido', 'Abalroamento no Mesmo Sentido', 'Abalroamento no mesmo sentido',
    'abalr. Transversal', 'Abalroamento Mesmo Sentido', 'abalr. Sentido Oposto',
    'Abalroamento sentido Oposto', 'Abal.mesmo Sentido', 'Abalroamento em Sent.Oposto',
    'Abalrroamento Transversal.', 'Abalroamento transversal.', 'Abalroamente Transversal',
    'Abalroamento Mesmo Sentido', 'abalr. Sentido Oposto', 'Abalroamento em Sent.Oposto',
    'Abalrroamento Transversal.', 'Abalromanemto mesmo sentido', 'Abalroamento Trarnversal',
    'Abalroamento Sentido Oposto', 'Abalr.Sentido Oposto', 'Abalr.Sentido Oposto.',
    'Abalr.Mesmo sentido', 'Abal.sentido. Oposto', 'Abal.sent.oposto', 'Abalr.sentido oposto', 'Abalr.Transversal', 'Abalr.mesmo sentido',
    'Abal. Sentido Oposto', 'Abalr. Sentido Oposto.',
    'Abalr.Mesmo sentido', 'Abalroamanento Traversasl', 'Abalr. Mesmo sentido', 'Abalr.em mesmo sentido',
    'Abalroamento no mesmo Sentido', 'abalr.mesmo Sentido',
    'Abalromanemto mesmo sentido', 'Abalroamento Trarnversal', 'Abalr.Sent.Oposto', 'Abalroamento em sentido oposto', 'Abalroamento em mesmo sentido', 'Abalroamento - Sentido Oposto', 'Ablr.Sentido Oposto',
    'Abalroamento transversal', 'Abalr. Mesmo Sentido', 'Abalroamento Longitudinal',  'Abalroamento Mesmo Sentido ',   'Abalr. Transversal', 'Abalroamento Transversal', 'Abalroamento mesmo sentido', 'Abal.mesmo sentido', 'Abalroamento  no mesmo sentido',     'Abalr. Sentido oposto'
]

# Lista com outros elementos
lista_outros = [
    'Outros', 'Outros - Sequência', 'Outros tipos de acidente', 'Outros Tipos de Acidente',
    'outros', 'Outros Tipos', 'Acidentes de outra natureza', 'Acidente - Outra Natureza',
    'acidente c/ morte', 'Danos Eventuais', 'Teste Wanderson', 'Acidente',
    'Queda de carga sobre a pista', 'Sem vítimas', 'Sem vítimas ', 'Com vítimas',  'Veículo em Chamas', 'Saída de pista', 'Objeto lançado contra veículo',
    None, 'Acidente c/ vitima', 'Acidente s/ vítima', 'Carreta deu L',
     'Sequência',  'Submersão', 'Outros tipos de Acidente', 'Outros Tipos de acidente',
    'Desatrelamento',
    'Outros(especificar)', 'Travessia Canteiro Central',
    'Outros - sequências', 'OUTROS', ' INCIDENTE - Choque na Praça - Cancela', 'Explosão', 'ponte ou viaduto',    'Sul', 'Norte', 'Incêndio', 'Transposição de Pista', 'derrapagem', 'Incidente', 'Derramamento de Carga', 'Outros - seguências',  'Imprudência', 'Não Def.', 'Problema Mecanico / Eletrico', 'Não Localizado / Evadiu-se'
]


capotamentos = ['Capotamento', 'CAPOTAMENTO',  'capotamento',]

# Engavetamentos
engavetamentos = ['ENGAVETAMENTO', 'engavetamento', 'Engavetamento']

# Tombamentos
tombamentos = ['Tombamento', 'TOMBAMENTO', 'Tombamento de moto', 'Tombamento de Moto', 'tombamento']

# Categoria Desconhecida
categoria_desconhecida = [' ']


In [14]:
def tipo_acidente(valor):
    if valor in colisoes:
        return "Colisão"
    elif valor in quedas:
        return "Queda"
    elif valor in saidas_de_pista:
        return "Saida de pista"
    elif valor in atropelamentos:
        return "Atropelamento"
    elif valor in choques_e_colisoes_com_objetos:
        return "Choques com objetos"
    elif valor in lista_outros:
        return "Outros"
    elif valor in lista_abalroamento:
      return "Abalroamento"
    elif valor in engavetamentos:
        return "Engavetamento"
    elif valor in tombamentos:
        return "Tombamento"
    elif valor in capotamentos:
        return "Capotamento"
    else:
        return valor

df['tipo_de_acidente'] = df['tipo_de_acidente'].apply(tipo_acidente)

# Valores registrados depois do tratamento
df['tipo_de_acidente'].value_counts()

<ipython-input-14-54931c872521>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tipo_de_acidente'] = df['tipo_de_acidente'].apply(tipo_acidente)


Colisão                10812
Choques com objetos     4756
Saida de pista          1773
Queda                   1766
Outros                  1660
Atropelamento           1395
Capotamento             1387
Engavetamento           1344
Tombamento               923
Abalroamento             357
Name: tipo_de_acidente, dtype: int64

#### Coluna 'sentido'

In [15]:
# Possiveis valores registrados

df['sentido'].value_counts()

Norte                      5723
Sul                        5602
Pista Norte                3665
Pista Sul                  3658
Decrescente                3445
Crescente                  3417
Leste                       216
Oeste                       180
NORTE                        97
SUL                          69
N                            26
Ambos                        16
S                            12
Sul - TRECHO CONTORNO         8
 Norte                        6
 Sul                          4
Norte - TRECHO CONTORNO       2
Name: sentido, dtype: int64

In [16]:
# Mudar tipografia de valores existentes para um padrão único
norte = ['Norte', 'Pista Norte', 'NORTE', 'N', 'Norte - TRECHO CONTORNO', ' Norte']
sul = ['Sul', 'Pista Sul', 'SUL', 'S', 'Sul - TRECHO CONTORNO', ' Sul']
leste = ['Leste']
oeste = ['Oeste']

def sentido(valor):
    if valor in norte:
        return "Norte"
    elif valor in sul:
        return "Sul"
    elif valor in leste:
        return "Leste"
    elif valor in oeste:
        return "Oeste"
    else:
        return valor

df['sentido'] = df['sentido'].apply(sentido)

# Valores registrados depois do tratamento
df['sentido'].value_counts()

<ipython-input-16-1a8274a49eb6>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentido'] = df['sentido'].apply(sentido)


Norte          9519
Sul            9353
Decrescente    3445
Crescente      3417
Leste           216
Oeste           180
Ambos            16
Name: sentido, dtype: int64

## Passa dados qualitativos para quantitativos

In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [18]:
#Função que irá verificar o que foi feito pelo LabelEncoder

def verifica_label_encoder(coluna):
  lista = []
  for i in coluna:
    if i not in lista:
      lista.append(i)
  for j in lista:
    rótulo_correspondente = le.inverse_transform([j])
    print(f"Valor Numérico {j} corresponde ao rótulo '{rótulo_correspondente[0]}'")

#### Gravidade

In [19]:
df['gravidade'] = le.fit_transform(df['gravidade_label'])
df.drop(['gravidade_label'], axis=1, inplace=True)

<ipython-input-19-957ce5132974>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gravidade'] = le.fit_transform(df['gravidade_label'])
<ipython-input-19-957ce5132974>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['gravidade_label'], axis=1, inplace=True)


In [20]:
#Como ficaram os dados de gravidade
verifica_label_encoder(df['gravidade'])

Valor Numérico 1 corresponde ao rótulo 'Leve'
Valor Numérico 2 corresponde ao rótulo 'Moderado'
Valor Numérico 3 corresponde ao rótulo 'Significativo'
Valor Numérico 0 corresponde ao rótulo 'Grave'


#### Tipo de acidente

In [21]:
df.drop(df[df['tipo_de_acidente'] == 'nan' ].index, inplace=True)
df.drop(df[df['tipo_de_acidente'] == ' ' ].index, inplace=True)
df = df.dropna(subset=["tipo_de_acidente"])

<ipython-input-21-532859e0e6f7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df['tipo_de_acidente'] == 'nan' ].index, inplace=True)
<ipython-input-21-532859e0e6f7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df['tipo_de_acidente'] == ' ' ].index, inplace=True)


In [22]:
df['tipo_de_acidente'].value_counts()

Colisão                10812
Choques com objetos     4756
Saida de pista          1773
Queda                   1766
Outros                  1660
Atropelamento           1395
Capotamento             1387
Engavetamento           1344
Tombamento               923
Abalroamento             357
Name: tipo_de_acidente, dtype: int64

In [23]:
df['tipo_acidente_quant'] = le.fit_transform(df['tipo_de_acidente'])
df.drop(['tipo_de_acidente'], axis=1, inplace=True)

<ipython-input-23-1adc60baada9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tipo_acidente_quant'] = le.fit_transform(df['tipo_de_acidente'])
<ipython-input-23-1adc60baada9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['tipo_de_acidente'], axis=1, inplace=True)


In [24]:
#Como ficaram os dados de tipo de acidente
verifica_label_encoder(df['tipo_acidente_quant'])

Valor Numérico 4 corresponde ao rótulo 'Colisão'
Valor Numérico 7 corresponde ao rótulo 'Queda'
Valor Numérico 8 corresponde ao rótulo 'Saida de pista'
Valor Numérico 3 corresponde ao rótulo 'Choques com objetos'
Valor Numérico 6 corresponde ao rótulo 'Outros'
Valor Numérico 1 corresponde ao rótulo 'Atropelamento'
Valor Numérico 2 corresponde ao rótulo 'Capotamento'
Valor Numérico 0 corresponde ao rótulo 'Abalroamento'
Valor Numérico 9 corresponde ao rótulo 'Tombamento'
Valor Numérico 5 corresponde ao rótulo 'Engavetamento'


#### Tipo de ocorrencia

In [25]:
df.drop(df[df['tipo_de_ocorrencia'] == 'nan'].index, inplace=True)
df = df.dropna(subset=["tipo_de_ocorrencia"])

<ipython-input-25-cf468e83800e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df['tipo_de_ocorrencia'] == 'nan'].index, inplace=True)


In [26]:
df['tipo_de_ocorrencia'].value_counts()

Sem vítimas        17049
Com vítimas         8819
Danos materiais      305
Name: tipo_de_ocorrencia, dtype: int64

In [27]:
df['tipo_ocorrencia_quant'] = le.fit_transform(df['tipo_de_ocorrencia'])
df.drop(['tipo_de_ocorrencia'], axis=1, inplace=True)

In [28]:
#Como ficaram os dados de tipo de acidente
verifica_label_encoder(df['tipo_ocorrencia_quant'])

Valor Numérico 2 corresponde ao rótulo 'Sem vítimas'
Valor Numérico 0 corresponde ao rótulo 'Com vítimas'
Valor Numérico 1 corresponde ao rótulo 'Danos materiais'


#### Sentido

In [29]:
df.drop(df[df['sentido'] == 'nan'].index, inplace=True)
df = df.dropna(subset=["sentido"])

In [30]:
df['sentido'].value_counts()

Norte          9510
Sul            9347
Decrescente    3445
Crescente      3417
Leste           211
Oeste           172
Ambos            16
Name: sentido, dtype: int64

In [31]:
df['sentido_quant'] = le.fit_transform(df['sentido'])
df.drop(['sentido'], axis=1, inplace=True)

<ipython-input-31-15677a6e043b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentido_quant'] = le.fit_transform(df['sentido'])
<ipython-input-31-15677a6e043b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['sentido'], axis=1, inplace=True)


In [32]:
#Como ficaram os dados de sentido
verifica_label_encoder(df['sentido_quant'])

Valor Numérico 4 corresponde ao rótulo 'Norte'
Valor Numérico 6 corresponde ao rótulo 'Sul'
Valor Numérico 3 corresponde ao rótulo 'Leste'
Valor Numérico 5 corresponde ao rótulo 'Oeste'
Valor Numérico 1 corresponde ao rótulo 'Crescente'
Valor Numérico 2 corresponde ao rótulo 'Decrescente'
Valor Numérico 0 corresponde ao rótulo 'Ambos'


#### Dataframe no final do processamento

In [33]:
df

,trecho,gravidade,tipo_acidente_quant,tipo_ocorrencia_quant,sentido_quant
0,BR-393/RJ,1,4,2,4
2,BR-290/RS,1,4,2,4
3,BR-116/RS,1,7,2,4
4,BR-290/RS,1,7,2,4
5,BR-290/RS,1,8,2,6
...,...,...,...,...,...
81421,BR-101/ES,1,4,2,4
81422,BR-101/ES,1,4,2,6
81425,BR-116/PR,1,3,2,6
81439,BR-101/ES,1,3,2,6


## Envia dados para aws

In [34]:
df

,trecho,gravidade,tipo_acidente_quant,tipo_ocorrencia_quant,sentido_quant
0,BR-393/RJ,1,4,2,4
2,BR-290/RS,1,4,2,4
3,BR-116/RS,1,7,2,4
4,BR-290/RS,1,7,2,4
5,BR-290/RS,1,8,2,6
...,...,...,...,...,...
81421,BR-101/ES,1,4,2,4
81422,BR-101/ES,1,4,2,6
81425,BR-116/PR,1,3,2,6
81439,BR-101/ES,1,3,2,6


In [35]:
%pip install python-dotenv

In [ ]:
% pip install sqlalchemy

In [41]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
import os


url = os.getenv("URL")
engine = create_engine(url)

Session = sessionmaker(bind=engine)
session = Session()

Base = declarative_base()

class SuaTabela(Base):
    __tablename__ = 'crashes'
    id = Column(Integer, primary_key=True)
    gravidade = Column(Integer)
    tipo_acidente_quant = Column(Integer)
    tipo_ocorrencia_quant	 = Column(Integer)
    sentido_quant = Column(Integer)

Base.metadata.create_all(engine)

session.close()

<ipython-input-41-7fff9aa53812>:13: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [42]:
df.to_sql('crashes', url, if_exists='replace', index=False) #Envia dados para tabela na aws

118